In [1]:
import turtle
import math

In [2]:
phi = 1.57/2
mat = [[math.cos(phi), -math.sin(phi)],
      [math.sin(phi), math.cos(phi)]]

def create_matrix(h, w):
    return [[0]*w for _ in range(h)]

def mul(l, r):
    res = create_matrix(len(l), len(r[0]))
    for p in range(len(l)):
        for q in range(len(r[0])):
            val = 0
            for ind in range(len(l[0])):
                val += l[p][ind] * r[ind][q]
            res[p][q] = val
    return res

def create_vector(vec):
    res = create_matrix(len(vec), 1)
    for ind in range(len(vec)):
        res[ind][0] = vec[ind]
    return res

def mat2tup(mat):
    return tuple(mat[t][0] for t in range(len(mat)))

def transpose(mat):
    h, w = len(mat), len(mat[0])
    tm = create_matrix(w, h)
    for p in range(h):
        for q in range(w):
            tm[q][p] = mat[p][q]
    return tm

def create_mnk_matrix(dat):
    b = create_matrix(len(dat), 2)
    y = create_matrix(len(dat), 1)
    for p in range(len(dat)):
        b[p][0] = dat[p][0]
        b[p][1] = 1
        y[p][0] = dat[p][1]
    return b, y

def copy(mat):
    return [t.copy() for t in mat]

def det2(m):
    return m[0][0]*m[1][1] - m[1][0]*m[0][1]

def subst(m, v, ind):
    ret = copy(m)
    for p in range(len(m)):
        ret[p][ind] = v[p][0]
    return ret

def cramers_solve(mat, vec):
    d = det2(mat)
    dx = det2(subst(mat, vec, 0))
    dy = det2(subst(mat, vec, 1))
    return dx/d, dy/d

In [9]:
def draw_points(a):
    turtle.up()
    turtle.clear()
    turtle.speed(0)
    turtle.delay(0.1)

    b, y = create_mnk_matrix(a)
    ys = mul(transpose(b), y)
    bs = mul(transpose(b), b)
    k, c = cramers_solve(bs, ys)

    for point in a:
        turtle.pencolor((0, 0, 0))
        turtle.goto(*point)
        turtle.dot()
        '''
        point1 = mat2tup(mul(mat, create_vector(point)))
        print('point1=', point1)
        turtle.goto(*point1)
        turtle.pencolor((1, 0, 0))
        turtle.dot()
        '''

    minx = min(t[0] for t in a)
    maxx = max(t[0] for t in a)
    d = maxx - minx

    turtle.color((0, 1, 0))
    for x in range(int(minx - d/2), int(maxx + d/2)):
        y=k*x+c
        turtle.goto(x, y)
        turtle.down()

In [10]:
class MyPoints(object):
    def __init__(self):
        self.dat = []
    def my_click(self,x,y):
        self.dat.append((x,y))
        if len(self.dat) > 1:
            draw_points(self.dat)

In [16]:
points = MyPoints()

In [18]:
turtle.forward(10)

In [19]:
turtle.getscreen().onclick(points.my_click)

In [ ]:
turtle.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.5/tkinter/__init__.py", line 1553, in __call__
    return self.func(*args)
  File "/usr/lib/python3.5/turtle.py", line 675, in eventfun
    fun(x, y)
  File "<ipython-input-10-6a81b1d4aeb5>", line 7, in my_click
    draw_points(self.dat)
  File "<ipython-input-9-5cd6a207cf32>", line 32, in draw_points
    turtle.down()
  File "<string>", line 5, in down
turtle.Terminator
Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.5/tkinter/__init__.py", line 1553, in __call__
    return self.func(*args)
  File "/usr/lib/python3.5/turtle.py", line 675, in eventfun
    fun(x, y)
  File "<ipython-input-10-6a81b1d4aeb5>", line 7, in my_click
    draw_points(self.dat)
  File "<ipython-input-9-5cd6a207cf32>", line 32, in draw_points
    turtle.down()
  File "<string>", line 5, in down
turtle.Terminator
Exception in Tkinter callback
Traceback (most recent call last):
  File 